In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
rootdir = os.getcwd()
os.chdir(rootdir)

In [4]:
gyrFile = 'Gyroscope'
accFile = 'Accelerometer'

normal = 'normal'

upstairs = 'upstairs'
downstairs = 'downstairs'

s1 = 'Smartphone1'
s3 = 'Smartphone3'

gaitpos = ['Normal', 'Upstairs', 'Downstairs']

# defining arrays for gyroscope data
gyrAssNormal_s1 = []
gyrAssUpstairs_s1 = []
gyrAssDownstairs_s1 = []

gyrAssNormal_s3 = []
gyrAssUpstairs_s3 = []
gyrAssDownstairs_s3 = []

#defining arrays for accelerometer data
accAssNormal_s1 = []
accAssUpstairs_s1 = []
accAssDownstairs_s1 = []

accAssNormal_s3 = []
accAssUpstairs_s3 = []
accAssDownstairs_s3 = []

# Parsing Gyroscope Normal Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [5]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if normal in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        gyrAssNormal_s1.append(gyrData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if normal in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        gyrAssNormal_s3.append(gyrData)                        
#End of file data import and processing

# Parsing Gyroscope Upstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [46]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if upstairs in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssUpstairs_s1.append(gyrData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if upstairs in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssUpstairs_s3.append(gyrData)                        
#End of file data import and processing

# Parsing Gyroscope Downstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if downstairs in subdir:
            for file in files:
                if gyrFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssDownstairs_s1.append(gyrData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if downstairs in subdir:
            for file in files:
                if gyrFile in file:
                    #print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        gyrData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        gyrAssDownstairs_s3.append(gyrData)                        
#End of file data import and processing

# Parsing Accelerometer Normal Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [8]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if normal in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        accAssNormal_s1.append(accData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if normal in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        # Assemble the data into a tensor
                        accAssNormal_s3.append(accData)                        
#End of file data import and processing

# Parsing Accelerometer Upstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [9]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if upstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssUpstairs_s1.append(accData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if upstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssUpstairs_s3.append(accData)                        
#End of file data import and processing

# Parsing Accelerometer Downstairs Data (Smartphone 1 - Rear Right) & (Smartphone 3 - Front Right)

In [10]:
for subdir, dirs, files in os.walk(rootdir):
    if s1 in subdir:
        if downstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))

                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssDownstairs_s1.append(accData)

for subdir, dirs, files in os.walk(rootdir):
    if s3 in subdir:
        if downstairs in subdir:
            for file in files:
                if accFile in file:
                    # print(os.path.join(subdir, file))
                    
                    with open(os.path.join(subdir, file), 'r') as f: # open in read-only mode
                        accData = pd.read_csv(f)
                        
                        # Assemble the data into a tensor
                        accAssDownstairs_s3.append(accData)                        
#End of file data import and processing

In [25]:
gaitpos = ['Normal', 'Upstairs', 'Downstairs']

for j in gaitpos:
    for i in range(1, 4, 2):
        gyrl = eval('gyrAss'+str(j) +'_s'+ str(i))
        accl = eval('accAss'+str(j) +'_s'+ str(i))
        
        if np.shape(gyrl) == (10,) and np.shape(accl) == (10,):
            print("Aquired all data from the group")
        else:
            print("Gyroscope " + str(j)+" Smartphone " + str(i)+ ": ", format(np.shape(gyrl)))
            print("Accelerometer " + str(j)+" Smartphone " + str(i)+ ": ", format(np.shape(accl)))

    print("\n")

Gyroscope Normal Smartphone 1:  (6,)
Accelerometer Normal Smartphone 1:  (6,)
Gyroscope Normal Smartphone 3:  (6,)
Accelerometer Normal Smartphone 3:  (6,)


Gyroscope Upstairs Smartphone 1:  (8,)
Accelerometer Upstairs Smartphone 1:  (8,)
Gyroscope Upstairs Smartphone 3:  (8,)
Accelerometer Upstairs Smartphone 3:  (8,)


Gyroscope Downstairs Smartphone 1:  (8,)
Accelerometer Downstairs Smartphone 1:  (8,)
Gyroscope Downstairs Smartphone 3:  (8,)
Accelerometer Downstairs Smartphone 3:  (8,)




In [40]:
def gyrPlot(t, y, lbl):
    plt.plot(t, y, label=lbl)

    plt.xlabel('Time (s)')
    plt.ylabel('Angular velocity (rad/s)')
    plt.title('Gyroscope')
    plt.legend()
    plt.show()

def accPlot(t, y, lbl):
    plt.plot(t, y, label=lbl)

    plt.xlabel('Time (s)')
    plt.ylabel('Acceleration (m/s^2)')
    plt.title('Accelerometer')
    plt.legend()
    plt.show()

gyrPlot(gyrAssUpstairs_s1[0]['Time (s)'], gyrAssUpstairs_s1[0]['X (rad/s)'], 'Smartphone 1')

KeyError: 'X (rad/s)'

In [13]:
acc1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
acc2 = ["John", "Doe", "Example", "code"]
c = [1, 2]

for i in c:
    something = 'acc'+str(i)
    # convert string to list
    print(something[0])

a
a
